In [1]:
!pip install transformers
!pip install datasets
!pip install numpy
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
Labels=pd.read_csv('/content/drive/MyDrive/Labels.txt',header=None, skipinitialspace=True)
Posts=pd.read_csv('/content/drive/MyDrive/Posts.txt',header=None, skipinitialspace=True)

,0
0,Free
1,Free
2,Free
3,Free
4,Hate
5,Free
6,Free
7,Free
8,Hate
9,Free


,0
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደረ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
5,በአማራ ህዝብ ግን ደም የገበረባትአጥንት የከሰከሰባት ሰንደቅአላማ ናትና ...
6,የኢትዮጵያ ብሄራዊ ቡድን በመቀሌ እስታዲየም ሲጫወት የገቡት ደጋፊዎች በፍ...
7,ዛሬም የአርሰናሉን ፔፔን የቶተንሀሙን ሰርጅ አውሪር ይዞ የገባው ኮትዲሾዋ...
8,ስማ ለምን ኡነተኛ ታሪክ አታወራም ለምንስ የሰው ታሪክ ትሰርቃላቹ የኢትዮ...
9,ሀገራዊ ወግ ያለው ለሁሉ የሆነ አስገድዶ ሳይሆን በፍቅርና በልዩ ሰዋዊ አ...


In [30]:
Frames = [Labels, Posts]
Data = pd.concat(Frames, axis=1)
Data.columns = ["labels", "posts"]
Data['labels'] = Data['labels'].replace(['Free', 'Free ','Hate'],[0,0,1])

config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.9996626377105713}]

In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import datasets

train_dataset, test_dataset = train_test_split(Data, test_size=0.20, random_state=50)
train_dataset, evaluate_dataset = train_test_split(Data, test_size=0.8, random_state=50)
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")

merged_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

In [5]:
training_data_size = merged_dataset['train'].num_rows
testing_data_size = merged_dataset['test'].num_rows

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")
def tokenize_function(data):
    return tokenizer(data["posts"], padding="max_length", truncation=True)
tokenized_datasets = merged_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets.set_format("torch")

In [11]:
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric

model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=2)
metric = load_metric("f1","accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.6 MB/s eta 0:00:00


In [1]:
exit()

In [9]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy
from torch.utils.data import DataLoader

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset= tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.train()




ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`